# Emerging Technologies Assessment

In [1]:
import random
from itertools import product

## Problem 1

In [2]:
def random_constant_balanced():
    """
    Returns a randomly chosen function that is either constant or balanced.
    
    The returned function takes four Boolean arguments as inputs and returns
    a single Boolean output.
    
    Returns:
        function: A function f(a, b, c, d) -> bool that is either constant
                  (always returns the same value) or balanced (returns True
                  for exactly half of the 16 possible input combinations).
    """
    # Randomly decide whether to create a constant or balanced function.
    # random.choice() selects one element uniformly at random from the list.
    # See: https://docs.python.org/3/library/random.html#random.choice
    function_type = random.choice(['constant', 'balanced'])
    
    # Check if we should create a constant function.
    if function_type == 'constant':
        # Randomly choose to return always True or always False.
        # This gives equal probability to both constant functions.
        constant_value = random.choice([True, False])
        
        # Define the constant function as a closure.
        def constant_function(a, b, c, d):
            """Constant function that always returns the same value."""
            # Return the predetermined constant value.
            return constant_value
        
        # Return the constant function.
        return constant_function
    
    # Otherwise, create a balanced function.
    else:  # balanced
        # Generate all 16 possible input combinations.
        # product([False, True], repeat=4) creates all 2^4 binary combinations.
        # See: https://docs.python.org/3/library/itertools.html#itertools.product
        all_inputs = list(product([False, True], repeat=4))
        
        # Randomly select 8 combinations that will return True.
        # random.sample() ensures no duplicates and uniform distribution.
        # See: https://docs.python.org/3/library/random.html#random.sample
        # Convert to set for O(1) membership testing later.
        true_inputs = set(random.sample(all_inputs, 8))
        
        # Define the balanced function as a closure.
        def balanced_function(a, b, c, d):
            """Balanced function that returns True for exactly half of inputs."""
            # Check if the input tuple is in the set of inputs that return True.
            # Set membership testing is O(1) on average.
            return (a, b, c, d) in true_inputs
        
        # Return the balanced function.
        return balanced_function

### Implementation Design Decisions

The function above uses several key Python programming concepts that are worth discussing in detail:

#### 1. Closures as Function Factories

The implementation uses [Python closures](https://realpython.com/inner-functions-what-are-they-good-for/#closures-and-factory-functions) to create functions dynamically. A closure is a function object that has access to variables in its enclosing lexical scope, even after the outer function has finished executing. In our case:

- For constant functions, the inner `constant_function` captures the `constant_value` variable
- For balanced functions, the inner `balanced_function` captures the `true_inputs` set

This pattern is called a "factory function" - a function that creates and returns other functions. This approach is elegant and follows functional programming principles, making the code more modular and easier to test.

#### 2. Time and Space Complexity

The complexity analysis of this implementation reveals interesting trade-offs:

**Constant Functions:**
- **Generation time**: O(1) - just need to randomly choose a Boolean value
- **Evaluation time**: O(1) - always return the same value
- **Space**: O(1) - only stores a single Boolean value

**Balanced Functions:**
- **Generation time**: O(2^n) where n=4, so O(16) - need to generate all input combinations
- **Evaluation time**: O(1) average case - set membership testing using hash tables
- **Space**: O(2^n) = O(16) - stores the set of "True" inputs

For larger values of $n$, this approach would become impractical. Alternative implementations might use:
- **Explicit formulas**: Some balanced functions (like parity functions) can be computed with simple formulas, requiring O(1) space
- **Lazy generation**: Generate balanced outputs on-the-fly using a seeded random number generator
- **Compressed representations**: Use mathematical properties to represent balanced functions more compactly

However, for $n=4$ (16 inputs), our direct approach is simple, clear, and efficient enough.

#### 3. Why Use Sets for Membership Testing?

We convert the list from `random.sample()` to a set because sets provide O(1) average-case lookup time using [hash tables](https://docs.python.org/3/tutorial/datastructures.html#sets), while lists require O(n) linear search. Given that we'll potentially call the balanced function many times during testing, this optimization is worthwhile. The trade-off is slightly higher memory usage (sets have more overhead than lists), but this is negligible for only 8 elements.

In [3]:
### Testing a Single Generated Function

# Generate a random function
f = random_constant_balanced()

# Test it on all 16 possible input combinations
print("Testing a randomly generated function on all inputs:\n")
print("Input (a, b, c, d)          | Output")
print("-" * 40)

true_count = 0
for inputs in product([False, True], repeat=4):
    result = f(*inputs)
    if result:
        true_count += 1
    # Format inputs as 0s and 1s for readability
    input_str = ''.join('1' if x else '0' for x in inputs)
    print(f"({inputs[0]!s:5}, {inputs[1]!s:5}, {inputs[2]!s:5}, {inputs[3]!s:5}) | {result}")

print("-" * 40)
print(f"\nSummary: Function returns True for {true_count} out of 16 inputs")

Testing a randomly generated function on all inputs:

Input (a, b, c, d)          | Output
----------------------------------------
(False, False, False, False) | True
(False, False, False, True ) | False
(False, False, True , False) | True
(False, False, True , True ) | True
(False, True , False, False) | False
(False, True , False, True ) | True
(False, True , True , False) | False
(False, True , True , True ) | False
(True , False, False, False) | False
(True , False, False, True ) | False
(True , False, True , False) | False
(True , False, True , True ) | True
(True , True , False, False) | False
(True , True , False, True ) | True
(True , True , True , False) | True
(True , True , True , True ) | True
----------------------------------------

Summary: Function returns True for 8 out of 16 inputs


In [4]:
# Determine the function type based on the count
if true_count == 0:
    print("Classification: CONSTANT (always False)")
elif true_count == 16:
    print("Classification: CONSTANT (always True)")
elif true_count == 8:
    print("Classification: BALANCED")
else:
    print(f"ERROR: Invalid function - returns True {true_count} times")

Classification: BALANCED


In [8]:
### Testing Multiple Functions

# Generate and classify several random functions to demonstrate variety
print("Generating 10 random functions and classifying them:\n")

for i in range(10):
    f = random_constant_balanced()
    
    # Count how many times the function returns True
    true_count = sum(f(*inputs) for inputs in product([False, True], repeat=4))
    
    # Classify the function
    if true_count == 0:
        function_type = "constant (always False)"
    elif true_count == 16:
        function_type = "constant (always True)"
    elif true_count == 8:
        function_type = "balanced"
    else:
        function_type = f"ERROR: returns True {true_count} times"
    
    print(f"Function {i+1:2d}: {function_type}")

Generating 10 random functions and classifying them:

Function  1: balanced
Function  2: constant (always True)
Function  3: constant (always True)
Function  4: balanced
Function  5: constant (always False)
Function  6: balanced
Function  7: balanced
Function  8: constant (always False)
Function  9: constant (always True)
Function 10: balanced


## Problem 2

## Problem 3

## Problem 4

## Problem 5